In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data_folder = f"../data"

sample_filename = "muestreos_parcelas.parquet"

df_samples = pd.read_parquet(f"{data_folder}/{sample_filename}")

df_samples["fecha"] = pd.to_datetime(df_samples["fecha"])


df_samples["codparcela"] = df_samples["codparcela"].str.replace("/", "")

df_samples.info()

<class 'pandas.core.frame.DataFrame'>
Index: 581793 entries, 0 to 581792
Data columns (total 61 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   generated_muestreos                                 581793 non-null  datetime64[us]
 1   codparcela                                          581793 non-null  object        
 2   provincia                                           581793 non-null  string        
 3   municipio                                           581793 non-null  string        
 4   fecha                                               581793 non-null  datetime64[us]
 5   campaña                                             581793 non-null  uint16        
 6   poligono                                            581731 non-null  UInt16        
 7   parcela                                             581763 non-null  UInt16        
 8  

In [5]:
parcela_filename = "parcelas_download.txt"
df_parcelas = pd.read_csv(f"{data_folder}/{parcela_filename}", sep="\t", usecols=["codparcela", "lat", "lon", "fecha_primera_muestra", "fecha_ultima_muestra"])
df_parcelas.head()

,codparcela,fecha_primera_muestra,fecha_ultima_muestra,lat,lon
0,015-00002-00-00,2006-03-08,2021-03-16,37.802069,-3.984598
1,043-00008-00-00,2006-04-10,2017-11-13,36.944813,-5.301941
2,017-00204-00-00,2006-05-10,2019-09-03,38.052148,-3.312451
3,010-00016-00-00,2006-03-06,2017-07-18,38.070431,-3.509408
4,007-00018-01-01,2010-03-08,2021-03-24,38.043972,-4.463010


In [25]:
#df_samples  = pd.merge(df_samples, df_parcelas, on='codparcela', how='left')
df_samples.head()

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,...,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada
0,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-16,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-27,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-04,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-11,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-17,2006,23,109,0,0,...,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
parcelas_with_data = [file.split("_BIOPAR")[0] for file in os.listdir("../data/raw/NVDI_FAPAR_data") if "NDVI" in file]

df_samples = df_samples[df_samples["codparcela"].isin(parcelas_with_data)]

In [61]:
df_samples.to_parquet("../data/subset_muestreos_parcelas.parquet")

In [8]:
raw_data_folder = f"../data/raw"
parcelas_with_data = [file.split("_BIOPAR")[0] for file in os.listdir(f"{raw_data_folder}/NVDI_FAPAR_data") if "NDVI" in file]


for id_parcela in parcelas_with_data:

    datos_parcela = [file for file in os.listdir(f"{raw_data_folder}/NVDI_FAPAR_data") if id_parcela in file]

    ndvi_filename = next((file for file in datos_parcela if "NDVI" in file), None)
    fapar_filename = next((file for file in datos_parcela if "FAPAR" in file), None)

    df_ndvi = pd.read_csv(f"{raw_data_folder}/NVDI_FAPAR_data/{ndvi_filename}", sep="\t", index_col=0)
    df_fapar = pd.read_csv(f"{raw_data_folder}/NVDI_FAPAR_data/{fapar_filename}", sep="\t", index_col=0)

    df_merge = pd.merge(df_ndvi, df_fapar, on='date', how='outer', suffixes=('_ndvi', '_fapar')).sort_values(by="date").reset_index(drop=True)

    df_merge.to_csv(f"../data/clean/NVDI_FAPAR_data/{id_parcela}.csv", sep="\t")

In [3]:
from zipfile import ZipFile
data_folder = f"../data/raw/other_variables_data"
unzip_data_folder = f"../data/clean/other_variables_data"

raw_files = os.listdir(data_folder)
nc_filename = "data.nc"

for raw_file in raw_files:
    filename = raw_file.split(".")[0]
    try:
        with ZipFile(f"{data_folder}/{raw_file}") as zip_object:
            zip_object.extractall(path=unzip_data_folder)

        os.rename(f"{unzip_data_folder}/{nc_filename}", f"{unzip_data_folder}/{filename}.nc")
    except:
        print(raw_file)

In [8]:

import os
unzip_data_folder = f"../data/clean/other_variables_data"

nc_files = os.listdir(unzip_data_folder)


In [9]:
unzip_data_folder

'../data/clean/other_variables_data'

In [2]:
import xarray as xr

# file_pattern = f"{unzip_data_folder}/METEO_DATA_*.nc"

# ds = xr.open_mfdataset(file_pattern, combine='by_coords', chunks={'time': 'auto'})


# Print the combined dataset

# Save the combined dataset to a new NetCDF file

#ds.to_netcdf("path/to/output/combined_data.nc")

In [11]:
ds.to_netcdf("../data/clean/combined_other_variables.nc")







In [3]:

ds = xr.open_dataset("../data/clean/combined_other_variables.nc")


In [83]:
df_parcelas = df_parcelas[df_parcelas.index>=3142]

In [86]:
df_parcelas

,codparcela,fecha_primera_muestra,fecha_ultima_muestra,lat,lon
3142,115-00087-00-/2,2006-04-11,2007-11-26,3.676176e+01,-6.004775e+00
3143,001-00006-00-00,2006-03-22,2006-11-21,3.750902e+01,-4.570450e+00
3144,008-00172-01-00,2013-03-26,2014-11-05,3.800243e+01,-3.568166e+00
3145,004-00212-00-00,2006-04-03,2007-11-12,3.746040e+01,-4.255956e+00
3146,018-00233-00-00,2006-04-11,2007-11-12,3.805630e+01,-3.319798e+00
...,...,...,...,...,...
5234,007-00086-01-01,2019-03-20,2019-03-20,3.731774e+01,-4.842136e+00
5235,035-00035-03-00,2021-03-11,2021-03-11,3.744146e+01,-4.647516e+00
5236,119-00019-00-A0,2008-10-27,2008-10-27,-9.999999e+06,-9.999999e+06
5237,035-00037-02-1,2021-03-15,2021-03-15,3.733449e+01,-3.860952e+00


In [6]:
for _, row in df_parcelas.iterrows():
    id_parcela = row["codparcela"].replace("/", "")

    start_time = row["fecha_primera_muestra"]
    end_time = row["fecha_ultima_muestra"]
    latitude_parcela = row["lat"]
    longitude_parcela = row["lon"]

    ds_parcela = ds.sel(time=slice(start_time, end_time))
    ds_parcela = ds_parcela.interp(latitude=latitude_parcela, longitude=longitude_parcela)

    df_data_parcela = ds_parcela.to_pandas()

    df_data_parcela.to_csv(f"../data/clean/other_variables_data_dfs/{id_parcela}.csv", sep="\t")

In [3]:
fapar_folder = "../data/clean/NVDI_FAPAR_data"
other_files_folder = "../data/clean/other_variables_data_dfs"
final_data_folder = "../data/clean/final"

fapar_files = os.listdir(fapar_folder)
other_files = os.listdir(other_files_folder)

In [4]:
for fapar_file in fapar_files:

    filename = fapar_file.split(".csv")[0]

    df_fapar = pd.read_csv(f"{fapar_folder}/{fapar_file}", sep="\t", index_col=0, parse_dates=["date"])

    df_other_files = pd.read_csv(f"{other_files_folder}/{fapar_file}", sep="\t", index_col=0, parse_dates=["time"]).reset_index()

    df_other_files.rename(columns={"time":"date"}, inplace=True)

    df_final_data = pd.merge(df_fapar, df_other_files, on="date", how="outer").sort_values(by="date").reset_index(drop=True)


    df_final_data.to_csv(f"{final_data_folder}/{fapar_file}", sep="\t")
